# Localisation d'objets brillants dans les images

Ce chapitre explore différentes méthodes pour localiser le centre d'objets brillants dans une image. Ces techniques sont couramment utilisées en astronomie (localisation d'étoiles, du soleil, de la lune) ou en microscopie (détection de cellules fluorescentes).

Nous comparerons plusieurs approches :
-   **Maximum simple** : Recherche du pixel le plus lumineux
-   **Barycentre d'intensité** : Centre de masse pondéré par l'intensité
-   **Ajustement gaussien** : Fitting d'une courbe gaussienne
-   **Seuillage et barycentre** : Méthode robuste basée sur un masque
-   **Ajustement de cercle** : Détection précise des contours circulaires

In [ ]:
# Importation des bibliothèques nécessaires
import imageio as iio  # Pour charger les images
import numpy as np
import matplotlib.pyplot as plt

# Chargement de l'image et conversion en niveaux de gris
# On utilise une image du soleil qui contient un objet brillant central
data = np.average(iio.imread('sun.jpg'), axis=-1)
# Alternative : image de la lune
# data = np.average(iio.imread('moon.png'), axis=-1)

---

## 1. Méthode du maximum simple

La méthode la plus simple consiste à trouver le pixel ayant la valeur d'intensité maximale :
-   **`argmax()`** : Retourne l'indice du maximum dans le tableau aplati
-   **`np.unravel_index()`** : Convertit l'indice 1D en coordonnées 2D (x, y)

**Limitations** : Cette méthode n'est pas robuste car :
-   Si plusieurs pixels ont la même valeur maximale, un seul est choisi arbitrairement
-   Sensible au bruit (un pixel aberrant peut être détecté comme le maximum)
-   Ne tient pas compte de la distribution spatiale de l'intensité

In [ ]:
# Recherche du pixel avec l'intensité maximale
im = plt.imshow(data, cmap='gray')

# data.argmax() : trouve l'indice du maximum dans le tableau aplati
# np.unravel_index() : convertit cet indice en coordonnées (x, y)
x, y = np.unravel_index(data.argmax(), data.shape)

# Affichage du point détecté en rouge
plt.plot([y], [x], marker='o', markersize=3, color="red")

print(y, x)
plt.show()

---

## 2. Méthode du barycentre d'intensité

Le barycentre (centre de masse) pondéré par l'intensité offre une localisation plus robuste :
-   **`np.where(data)`** : Retourne les coordonnées de tous les pixels non nuls
-   **`np.average(weights=...)`** : Calcule la moyenne pondérée des positions
-   Les pixels plus lumineux ont plus de poids dans le calcul

**Limitations** :
-   Nécessite une distinction claire entre l'objet et l'arrière-plan
-   Peut être biaisé si l'arrière-plan n'est pas uniforme
-   Sensible à la présence d'autres objets brillants

In [ ]:
# Calcul du barycentre pondéré par l'intensité
im = plt.imshow(data, cmap='gray')

# Copie et normalisation de l'image
tmp_data = data.copy()
tmp_data /= np.max(tmp_data)  # Normalisation entre 0 et 1

# Extraction des coordonnées de tous les pixels
pos = np.where(data)

# Calcul de la moyenne pondérée des positions
# Les pixels plus lumineux contribuent davantage au calcul
x = int(np.average(pos[0], weights=data[pos]))
y = int(np.average(pos[1], weights=data[pos]))

# Affichage du barycentre détecté
plt.plot([y], [x], marker='o', markersize=3, color="red")

print(y, x)
plt.show()

---

## 3. Méthode d'ajustement gaussien

Cette méthode suppose que la distribution d'intensité suit une forme gaussienne :
-   **Projection 1D** : Somme des intensités le long de chaque axe
-   **`optimize.curve_fit()`** : Ajuste une courbe gaussienne sur ces projections
-   **Paramètre `mu`** : Centre de la gaussienne ajustée

Cette approche fonctionne bien pour les objets très brillants avec une distribution d'intensité régulière (comme le soleil ou des sources lumineuses ponctuelles).

**Avantages** : Robuste au bruit, fonctionne avec des objets étendus

In [ ]:
# Ajustement d'une courbe gaussienne sur les projections X et Y
from scipy import optimize

# Définition de la fonction gaussienne
# A : amplitude, mu : centre, sigma : largeur
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

data = data.astype(float)

# Création d'une figure avec 2x2 sous-graphiques
fig, axs = plt.subplots(2, 2, figsize=(16, 10))
im = axs[0,0].imshow(data, cmap='gray')
axs[-1, -1].axis('off')  # Le dernier subplot n'est pas utilisé

# Projection et ajustement sur l'axe X (vertical)
x_ = np.arange(data.shape[0])
func_ = np.sum(data**2, axis=1)  # Somme des carrés pour accentuer les pics
axs[0,1].plot(func_, x_[::-1])  # Affichage inversé pour correspondre à l'image

# Ajustement de la gaussienne avec des paramètres initiaux
coeff_1, _ = optimize.curve_fit(gauss, x_, func_, p0=[1., 0., 1.])
axs[0,1].plot(gauss(x_, *coeff_1), x_[::-1])

# Projection et ajustement sur l'axe Y (horizontal)
x_ = np.arange(data.shape[1])
func_ = np.sum(data**2, axis=0)
axs[1,0].plot(x_, func_)
coeff_2, _ = optimize.curve_fit(gauss, x_, func_, p0=[1., 0., 1.])
axs[1,0].plot(x_, gauss(x_, *coeff_2))

# Les centres des gaussiennes donnent les coordonnées du centre
x, y = int(coeff_1[1]), int(coeff_2[1])
axs[0,0].plot([y], [x], marker='o', markersize=3, color="red")

fig.tight_layout()
print(y, x)
plt.show()

---

## 4. Méthode du seuillage et barycentre

Cette méthode combine seuillage et calcul de barycentre pour une détection plus robuste :
-   **Seuillage** : Création d'un masque binaire (pixels > seuil)
-   **`np.argwhere()`** : Extraction des coordonnées des pixels du masque
-   **`np.average()`** : Calcul du barycentre des pixels sélectionnés

**Avantages** :
-   Simple et efficace
-   Robuste si l'objet est suffisamment brillant
-   Élimine l'influence de l'arrière-plan

**Note** : Le choix du seuil est crucial pour la précision

In [ ]:
# Méthode de seuillage suivie du calcul de barycentre
im = plt.imshow(data, cmap='gray')

# Application d'un seuil pour isoler l'objet brillant
tmp_data = data.copy()
tmp_data[data < 150] = 0  # Mise à zéro des pixels faibles

# Calcul du barycentre du masque
# np.argwhere() : retourne les indices des pixels non nuls
# np.average() : calcule la moyenne des coordonnées
x, y = np.average(np.argwhere(tmp_data), axis=0).astype(int)

# Affichage du centre détecté
plt.plot([y], [x], marker='o', markersize=3, color="red")

print(y, x)
plt.show()

---

## 5. Méthode d'ajustement de cercle

Pour les objets circulaires (comme le soleil), on peut ajuster un cercle au masque :
-   **Masque binaire** : Seuillage avec `img_as_bool()`
-   **Fonction de coût** : Mesure la différence entre le cercle théorique et le masque
-   **`optimize.fmin()`** : Optimise les paramètres (centre x, y et rayon r)
-   **`draw.disk()`** : Génère un disque théorique pour la comparaison

Cette méthode est la plus précise pour les objets circulaires bien définis.

**Avantages** : Très précise, fournit également le rayon

In [ ]:
# Ajustement d'un cercle au masque de l'objet
from skimage import io, color, measure, draw, img_as_bool
import numpy as np

# Création d'un masque binaire par seuillage
tmp_data = data.copy()
tmp_data[data < 200] = 0
tmp_data = img_as_bool(tmp_data)  # Conversion en booléen

# Définition de la fonction de coût pour l'optimisation
# Minimise la différence entre le cercle théorique et le masque
def cost(params):
    x0, y0, r = params
    # Génération d'un disque théorique
    coords = draw.disk((x0, y0), r, shape=tmp_data.shape)
    template = np.zeros_like(tmp_data)
    template[coords] = 1
    # Calcul du nombre de pixels différents (coût négatif pour maximisation)
    return -np.sum(template == tmp_data)

# Estimation initiale du centre (barycentre du masque)
init_x, init_y = np.average(np.argwhere(tmp_data), axis=0)

# Optimisation des paramètres (x, y, rayon)
# p0 : estimation initiale (centre + rayon approximatif de 50)
x, y, r = optimize.fmin(cost, (init_x, init_y, 50))

# Affichage du résultat
x, y = int(x), int(y)
f, ax = plt.subplots()

# Dessin du cercle ajusté
circle = plt.Circle((y, x), r, fill=False, color='red', linewidth=2)
ax.imshow(data, cmap='gray')
ax.add_artist(circle)

# Affichage du centre
ax.plot([y], [x], marker='o', markersize=3, color="red")

print(y, x)
plt.show()